In [2]:
def readGrid(path):
    with open(f"../assets/data/DoubleMinesweeper/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("easy1_10x10")
    for g in grid:
        print(g)

['-', '2', '2', '3', '-', '-', '2', '2', '2', '-']
['-', '4', '-', '-', '4', '4', '-', '-', '2', '-']
['2', '-', '-', '-', '5', '-', '5', '4', '2', '2']
['-', '-', '3', '3', '4', '-', '4', '2', '-', '2']
['-', '2', '-', '3', '5', '-', '3', '2', '3', '3']
['2', '3', '2', '3', '-', '3', '1', '-', '4', '-']
['1', '-', '1', '4', '-', '5', '1', '4', '-', '-']
['3', '4', '2', '3', '-', '3', '-', '-', '-', '4']
['-', '3', '-', '-', '4', '5', '-', '-', '3', '-']
['-', '3', '1', '3', '-', '-', '1', '1', '2', '-']


In [8]:
from ortools.sat.python import cp_model as cp

def doubleminesweeper_solver(m, n, grid):
    model = cp.CpModel()
    x = dict()
    for i in range(m):
        for j in range(n):
            if grid[i][j] in "-.*":
                x[i, j, 1] = model.NewBoolVar(f"x_{i}_{j}_1")
                x[i, j, 2] = model.NewBoolVar(f"x_{i}_{j}_2")
                model.Add(x[i, j, 1] <= x[i, j, 2])
    for i in range(m):
        for j in range(n):
            ready_list = []
            if grid[i][j] not in "-.*":
                for x_ in range(-1, 2):
                    for y_ in range(-1, 2):
                        if (i + x_, j + y_, 1) in x:
                            ready_list.append(x[i + x_, j + y_, 1])
                            ready_list.append(x[i + x_, j + y_, 2])
                model.Add(sum(ready_list) == int(grid[i][j]))
                
    solver = cp.CpSolver()
    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        for i in range(m):
            for j in range(n):
                if grid[i][j] in "-.*":
                    if solver.Value(x[i, j, 1]) > 1e-1 and solver.Value(x[i, j, 2]) > 1e-1:
                        print("B", end=" ")
                        # 2 bombs
                    elif solver.Value(x[i, j, 1]) == 0 and solver.Value(x[i, j, 2]) == 0:
                        print("0", end = " ")
                    else:
                        print("b", end = " ")
                        # 1 bomb
                else:
                    print(grid[i][j], end = " ")
            print()
        print()
                
if __name__ == "__main__":
    m, n, grid = readGrid("easy1_10x10")
    doubleminesweeper_solver(m, n, grid)

0 2 2 3 b 0 2 2 2 B 
B 4 0 B 4 4 B 0 2 0 
2 0 B 0 5 b 5 4 2 2 
0 0 3 3 4 B 4 2 B 2 
b 2 b 3 5 b 3 2 3 3 
2 3 2 3 B 3 1 0 4 b 
1 b 1 4 0 5 1 4 B b 
3 4 2 3 B 3 b 0 b 4 
B 3 b 0 4 5 0 b 3 0 
0 3 1 3 B 0 1 1 2 b 

